<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_09_25_Connected_Data_Knowledge_Graph_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to generate a knowledge graph from extracted entities, based on the [Connected Data Knowledge Graph Challenge](https://github.com/Connected-Data/cdkg-challenge) dataset.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [2]:
!pip install --upgrade graphlit-client

Install Pyvis

In [3]:
!pip install --upgrade pyvis

Clone the original repo

In [4]:
!rm -rf cdkg-challenge
!git clone https://github.com/Connected-Data/cdkg-challenge

Cloning into 'cdkg-challenge'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 105 (delta 18), reused 4 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (105/105), 1.11 MiB | 1.54 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [5]:
DATA_DIRECTORY = "cdkg-challenge"

PATH_TRANSCRIPTS = DATA_DIRECTORY + "/Transcripts/"

Initialize Graphlit

In [6]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [44]:
import base64
import mimetypes
import asyncio
from typing import List, Optional
from tqdm.asyncio import tqdm

async def process_file(filename, workflow_id, progress_bar):
    print(f'Ingesting content from [{filename}]')

    content_id = await ingest_file(filename, workflow_id)

    if content_id is not None:
        print(f'Ingested content [{content_id}] from [{filename}].')
    else:
        print(f'Failed to ingest content from [{filename}].')

    progress_bar.update(1)

# NOTE: for local files, load from disk and convert to Base64 data
async def ingest_file(file_path: str, workflow_id: str):
    if graphlit.client is None:
        return;

    try:
        file_name = os.path.basename(file_path)
        content_name, _ = os.path.splitext(file_name)

        mime_type = mimetypes.guess_type(file_name)[0]

        if mime_type is None:
            print(f'Failed to infer MIME type [{file_name}], treat as plain text')
            mime_type = "text/plain"

        if mime_type == "text/csv":
            print(f'Skipping MIME type [{mime_type}]')
            return None

        if not os.path.exists(file_path):
            print(f'File [{file_path}] does not exist')
            return None

        with open(file_path, "rb") as file:
            file_content = file.read()

            base64_content = base64.b64encode(file_content).decode('utf-8')

            # Using synchronous mode, so the notebook waits for the content to be ingested
            response = await graphlit.client.ingest_encoded_file(name=content_name, data=base64_content, mime_type=mime_type, workflow=input_types.EntityReferenceInput(id=workflow_id), is_synchronous=True)

            return response.ingest_encoded_file.id if response.ingest_encoded_file is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

# Create specification for Anthropic Sonnet 3.5
async def create_anthropic_specification():
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name="Anthropic Claude Sonnet 3.5",
        type=enums.SpecificationTypes.EXTRACTION,
        serviceType=enums.ModelServiceTypes.ANTHROPIC,
        anthropic=input_types.AnthropicModelPropertiesInput(
            model=enums.AnthropicModels.CLAUDE_3_5_SONNET,
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

# Create specification for OpenAI GPT-4o
async def create_openai_specification():
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name="OpenAI GPT-4o",
        type=enums.SpecificationTypes.EXTRACTION,
        serviceType=enums.ModelServiceTypes.OPEN_AI,
        openAI=input_types.OpenAIModelPropertiesInput(
            model=enums.OpenAIModels.GPT4O_128K,
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

# Create entity extraction workflow using LLM specification
async def create_workflow(specification_id: str):
    if graphlit.client is None:
        return;

    input = input_types.WorkflowInput(
        name="Entity Extraction",
        extraction=input_types.ExtractionWorkflowStageInput(
            jobs=[
                input_types.ExtractionWorkflowJobInput(
                    connector=input_types.EntityExtractionConnectorInput(
                        type=enums.EntityExtractionServiceTypes.MODEL_TEXT,
                        modelText=input_types.ModelTextExtractionPropertiesInput(
                            specification=input_types.EntityReferenceInput(id=specification_id)
                        )
                    )
                )
            ]
        )
    )

    try:
        response = await graphlit.client.create_workflow(input)

        return response.create_workflow.id if response.create_workflow is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def query_contents_graph():
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.query_contents_graph(
            graph=input_types.ContentGraphInput(
                # Uncomment to filter just on software and organization nodes
#                types=[enums.ObservableTypes.SOFTWARE,enums.ObservableTypes.ORGANIZATION]
            )
        )

        return response.contents.graph if response.contents is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def query_contents_facets():
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.query_contents_facets(
            filter=input_types.ContentFilter(
                offset=0,
                limit=0,
            ),
            facets=[
                input_types.ContentFacetInput(
                    facet=enums.ContentFacetTypes.OBSERVABLE
                )
            ]
        )

        return response.contents.facets if response.contents is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

# NOTE: these functions are just used to clean-up old data before executing the example
async def delete_all_specifications():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_specifications(is_synchronous=True)

async def delete_all_contents():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_contents(is_synchronous=True)

async def delete_all_workflows():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_workflows(is_synchronous=True)


Define Graphlit knowledge graph helper functions. Required for rendering the knowledge graph with Pyvis.

In [38]:
import random
import os
import json
from pyvis.network import Network
from typing import Optional

def select_emoji(entity_type):
    # Emoji mappings for observable types
    observable_emoji_map = {
        "CONTENT": "📄",  # Page facing up emoji for generic content
        "LABEL": "🏷️",   # Label emoji for categories or tags
        "PERSON": "🧑",  # Person emoji for individuals
        "ORGANIZATION": "🏢",  # Office building emoji for organizations
        "PLACE": "🌍",  # Globe showing Europe-Africa for places
        "PRODUCT": "🛍️",  # Shopping bags emoji for products
        "SOFTWARE": "💻",  # Laptop emoji for software
        "REPO": "🗂️",  # Card index dividers emoji for repositories
        "EVENT": "🎉",  # Party popper emoji for events
        "MEDICAL_STUDY": "📊",  # Bar chart emoji for medical studies
        "MEDICAL_CONDITION": "🤒",  # Face with thermometer emoji for medical conditions
        "MEDICAL_GUIDELINE": "📜",  # Scroll emoji for medical guidelines
        "MEDICAL_DRUG": "💊",  # Pill emoji for medical drugs
        "MEDICAL_DRUG_CLASS": "🧬",  # DNA emoji for medical drug classes
        "MEDICAL_INDICATION": "🔍",  # Magnifying glass emoji for medical indications
        "MEDICAL_CONTRAINDICATION": "🚫",  # Prohibited emoji for contraindications
        "MEDICAL_TEST": "🧪",  # Test tube emoji for medical tests
        "MEDICAL_DEVICE": "🦾",  # Mechanical arm emoji for medical devices
        "MEDICAL_THERAPY": "🩺",  # Stethoscope emoji for medical therapies
        "MEDICAL_PROCEDURE": "🔧",  # Wrench emoji for medical procedures
    }

    # Return the emoji corresponding to the entity type
    return observable_emoji_map.get(entity_type, "📄")  # Default to page facing up emoji if entity type is unknown

def lookup_node_shape(entity_type, content_type, file_type):
    entity_icon_map = {
        "CONTENT": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf15c", "color": "#aec7e8"}},  # file-text
        "LABEL": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf02b", "color": "#ffbb78"}},   # tag (luggage tag-like)
        "PERSON": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf007", "color": "#98df8a"}},  # user
        "ORGANIZATION": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf1ad", "color": "#ff9896"}},  # building
        "PLACE": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf3c5", "color": "#c5b0d5"}},  # globe-americas
        "PRODUCT": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf1b2", "color": "#c49c94"}},  # cube
        "SOFTWARE": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf085", "color": "#f7b6d2"}},  # cog
        "REPO": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf1c0", "color": "#c7c7c7"}},  # database
        "EVENT": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf073", "color": "#dbdb8d"}},  # calendar
        "MEDICAL_STUDY": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf201", "color": "#17becf"}},  # bar-chart
        "MEDICAL_CONDITION": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf0f1", "color": "#d62728"}},  # heartbeat
        "MEDICAL_GUIDELINE": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf02d", "color": "#9467bd"}},  # book
        "MEDICAL_DRUG": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf484", "color": "#bcbd22"}},  # capsules
        "MEDICAL_DRUG_CLASS": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf471", "color": "#1f77b4"}},  # dna
        "MEDICAL_INDICATION": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf0ae", "color": "#2ca02c"}},  # search
        "MEDICAL_CONTRAINDICATION": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf05e", "color": "#ff7f0e"}},  # ban
        "MEDICAL_TEST": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf469", "color": "#e377c2"}},  # vial
        "MEDICAL_DEVICE": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf6a8", "color": "#7f7f7f"}},  # robot
        "MEDICAL_THERAPY": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf0f9", "color": "#8c564b"}},  # stethoscope
        "MEDICAL_PROCEDURE": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf0ad", "color": "#17becf"}},  # wrench
    }

    content_icon_map = {
        "FILE": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf15c", "color": "#aec7e8"}},  # file
        "PAGE": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf0ac", "color": "#aec7e8"}},  # globe
        "MESSAGE": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf075", "color": "#aec7e8"}},  # comment
        "TEXT": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf15c", "color": "#aec7e8"}},  # file-text
        "POST": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf1ea", "color": "#aec7e8"}},  # newspaper
        "EMAIL": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf0e0", "color": "#aec7e8"}},  # envelope
        "EVENT": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf073", "color": "#aec7e8"}},  # calendar
        "ISSUE": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf188", "color": "#aec7e8"}},  # bug
    }

    file_icon_map = {
        "VIDEO": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf03d", "color": "#aec7e8"}},  # video-camera
        "AUDIO": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf028", "color": "#aec7e8"}},  # volume-up
        "IMAGE": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf03e", "color": "#aec7e8"}},  # picture-o (image)
        "DOCUMENT": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf15b", "color": "#aec7e8"}},  # file-text-o
        "EMAIL": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf0e0", "color": "#aec7e8"}},  # envelope
        "CODE": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf121", "color": "#aec7e8"}},  # code
        "DATA": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf1c0", "color": "#aec7e8"}},  # database
    }

    if file_type is not None:
        return file_icon_map.get(file_type, content_icon_map.get("FILE"))
    elif content_type is not None:
        return content_icon_map.get(content_type, content_icon_map.get("FILE"))
    else:
        return entity_icon_map.get(entity_type, {"shape": "dot"})  # Default to a simple dot shape if the entity type is unknown

def lookup_node_color(entity_type):
    entity_color_map = {
        "CONTENT": "#aec7e8",  # Soft blue
        "LABEL": "#ffbb78",   # Soft orange
        "PERSON": "#98df8a",  # Pale green
        "ORGANIZATION": "#ff9896",  # Soft red
        "PLACE": "#c5b0d5",  # Soft purple
        "PRODUCT": "#c49c94",  # Soft brown
        "SOFTWARE": "#f7b6d2",  # Light pink
        "REPO": "#c7c7c7",  # Light gray
        "EVENT": "#dbdb8d",  # Soft yellow
        "MEDICAL_STUDY": "#17becf",  # Cyan
        "MEDICAL_CONDITION": "#d62728",  # Strong red
        "MEDICAL_GUIDELINE": "#9467bd",  # Medium purple
        "MEDICAL_DRUG": "#bcbd22",  # Olive green
        "MEDICAL_DRUG_CLASS": "#1f77b4",  # Strong blue
        "MEDICAL_INDICATION": "#2ca02c",  # Vivid green
        "MEDICAL_CONTRAINDICATION": "#ff7f0e",  # Vivid orange
        "MEDICAL_TEST": "#e377c2",  # Soft magenta
        "MEDICAL_DEVICE": "#7f7f7f",  # Medium gray
        "MEDICAL_THERAPY": "#8c564b",  # Muted brown
        "MEDICAL_PROCEDURE": "#17becf",  # Cyan (same as MEDICAL_STUDY)
    }

    return entity_color_map.get(entity_type, "#ffffff")  # Default to white if entity type is unknown

def parse_metadata(metadata):
    if metadata is None:
        return None, None

    o = json.loads(metadata)

    return enums.ContentTypes[o["type"]] if "type" in o else None, enums.FileTypes[o["fileType"]] if "fileType" in o else None

def pretty_print_json(dictionary):
    return '\n'.join(f"{key}: {value}" for key, value in dictionary.items() if not key.startswith('@'))

def parse_title(metadata):
    if metadata is None:
        return None

    o = json.loads(metadata)

    if o is not None:
        uri = o["uri"] if "uri" in o else None

        title = pretty_print_json(o)

        if uri is not None:
            return f'URI: {uri}' + '\n' + title
        else:
            return title
    else:
        return None

def parse_label(metadata):
    if metadata is None:
        return None

    o = json.loads(metadata)

    file_name = o["fileName"] if "fileName" in o else None

    label = None

    document = o["document"] if "document" in o else None
    audio = o["audio"] if "audio" in o else None
    video = o["video"] if "video" in o else None

    if document is not None and "title" in document:
        label = document["title"]
    elif video is not None and "title" in video:
        label = video["title"]
    elif audio is not None and "title" in audio:
        label = audio["title"]

    return label if label is not None else file_name

def format_relation(relation: str):
    if relation == "observed-by":
        return None

    return relation.replace("-", " ")

def create_pyvis_graph(graph):
    g = create_pyvis_network()

    if graph.nodes is not None:
        for node in graph.nodes:
            content_type = None
            file_type = None
            label = None
            title = None

            parsed_title = parse_title(node.metadata)

            if parsed_title is not None:
                if node.type == enums.EntityTypes.CONTENT:
                    content_type, file_type = parse_metadata(node.metadata)
                    label = parse_label(node.metadata)
                    title = f'{node.type.name} [{node.id}]\n' + parsed_title
                else:
                    title = f'{node.type.name} [{node.id}]\n' + parsed_title

            shape = lookup_node_shape(node.type.name, content_type, file_type)

            if shape is not None:
                g.add_node(node.id, label=label if label is not None else node.name, shape=shape["shape"], icon=shape.get("icon"), color=lookup_node_color(node.type.name), title=title if title is not None else f'{node.type.name} [{node.id}]')

    if graph.edges is not None:
        for edge in graph.edges:
            # ensure start and end vertex exist in graph
            if not edge.from_ in g.node_ids:
                g.add_node(edge.from_)
            if not edge.to in g.node_ids:
                g.add_node(edge.to)

            relation = format_relation(edge.relation)

            width = 3 if edge.relation != "observed-by" else 1

            g.add_edge(edge.from_, edge.to, label=relation, title=relation, width=width, arrowStrikethrough=False, arrows="middle")

    return g

def create_pyvis_network():
    g = Network(
        notebook=False,
        directed=True,
        cdn_resources="in_line",
        height="900px",
        width="100%",
    )

    return g

Execute Graphlit example

In [45]:
# Remove any existing specifications, workflows and contents; only needed for notebook example
await delete_all_workflows()
await delete_all_specifications()
await delete_all_contents()

print('Deleted all specifications, workflows and contents.')

# NOTE: depending on LLM rate limits, specify if ingestion/extraction should run in parallel or not
run_parallel = True

# NOTE: select the LLM to use for extraction
specification_id = await create_openai_specification()
#specification_id = await create_anthropic_specification()

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    workflow_id = await create_workflow(specification_id)

    if workflow_id is not None:
        print(f'Created workflow [{workflow_id}].')

        # Get all files in the directory and subdirectories
        files = [os.path.join(root, file) for root, _, files in os.walk(PATH_TRANSCRIPTS) for file in files]

        progress_bar = tqdm(total=len(files))

        if run_parallel:
            tasks = [process_file(filename, workflow_id, progress_bar) for filename in files]

            await asyncio.gather(*tasks)
        else:
            # NOTE: not running parallel, to keep within LLM rate limits
            for filename in files:
                await process_file(filename, workflow_id, progress_bar)

        print('Ingested all contents.')

Deleted all specifications, workflows and contents.
Created specification [20162969-c9d1-4892-9b7e-3edfa47b30da].
Created workflow [b7bc8378-02ea-40f6-91e4-dad3ff2e2efa].



100%|██████████| 55/55 [28:35<00:00, 31.20s/it]

Ingesting content from [cdkg-challenge/Transcripts/Connected Data Knowledge Graph Challenge - Transcript Metadata.csv]
Skipping MIME type [text/csv]
Failed to ingest content from [cdkg-challenge/Transcripts/Connected Data Knowledge Graph Challenge - Transcript Metadata.csv].
Ingesting content from [cdkg-challenge/Transcripts/Connected Data World 2021/Presentations/Omar Khan and David Newman - The Enterprise Knowledge Graph _ CDW21 Presentation.srt]
Ingesting content from [cdkg-challenge/Transcripts/Connected Data World 2021/Presentations/Dave Duggal - One graph to bind them all! Linking data & apps for event-driven interoperability.srt]
Ingesting content from [cdkg-challenge/Transcripts/Connected Data World 2021/Presentations/Jörg Schad - Graph Analytics vs Graph Machine Learning _ CDW21 Presentation.srt]
Ingesting content from [cdkg-challenge/Transcripts/Connected Data World 2021/Presentations/Hybridization of Machine Learning and Operational Research is the future of AI _ Nikolaj van



  4%|▎         | 2/55 [00:37<16:28, 18.65s/it]

Ingested content [eafebd0f-3bcb-4e93-b5fe-347e9a425bb7] from [cdkg-challenge/Transcripts/Connected Data World 2021/Presentations/Legal Knowledge Graphs _ Vaishali Raghvani _ Connected Data World.srt].


...

Ingested content [8d00e167-b0fb-4b8c-961d-f036325e042d] from [cdkg-challenge/Transcripts/Knowledge Connexions 2020/Presentations/Rebooting AI Adding Knowledge to Deep Learning.srt].



 96%|█████████▋| 53/55 [02:19<00:06,  3.27s/it]

Ingested content [d87b3d63-3783-46ac-a955-14287058348a] from [cdkg-challenge/Transcripts/2024/Connected Data London Roundtable July 3 2024.txt].



 98%|█████████▊| 54/55 [03:51<00:27, 27.01s/it]

HTTP status code: 500
Failed to ingest content from [cdkg-challenge/Transcripts/Knowledge Connexions 2020/Presentations/Illuminating the Last Vestiges of Dark Data - Speech Recognition for Knowledge Graphs.srt].
HTTP status code: 500
Failed to ingest content from [cdkg-challenge/Transcripts/Knowledge Connexions 2020/Presentations/BizDevOps Aligning the Business and IT with a Graph Knowledge Base.srt].
Ingested all contents.


In [46]:
from IPython.display import display, HTML, JSON

# Query the resulting knowledge graph
graph = await query_contents_graph()

if graph is not None:
    g = create_pyvis_graph(graph)

    g.set_options("""
    var options = {
        "physics": {
            "forceAtlas2Based": {
                "gravitationalConstant": -50,
                "centralGravity": 0.01,
                "springLength": 100,
                "springConstant": 0.08
            },
            "maxVelocity": 50,
            "solver": "forceAtlas2Based",
            "timestep": 0.35,
            "stabilization": {
                "iterations": 100
            }
        }
        }
    """)

    # render with random file name
    graph_html = g.generate_html(f"graph_{random.randint(0, 1000)}.html")

    # Inject FontAwesome CSS
    font_awesome_link = '<script src="https://kit.fontawesome.com/2c74303849.js" crossorigin="anonymous"></script>'
    graph_html = graph_html.replace('<head>', f'<head>{font_awesome_link}')

    display(HTML(graph_html))

Display histogram of extracted entities

In [47]:
import plotly.express as px
import pandas as pd

facets = await query_contents_facets()

if facets is not None:
    json_strings = [facet.model_dump_json(indent=2) for facet in facets if facet is not None]  # Using indent for pretty printing
    json_dicts = [json.loads(js) for js in json_strings]

    df = pd.json_normalize(json_dicts, sep='_')

    # Rename columns for clarity
    df.rename(columns={
        'observable_type': 'Type',
        'observable_observable_name': 'Name',
        'count': 'Count'
    }, inplace=True)

    # Sort the DataFrame by 'Name' alphabetically
    df = df.sort_values(by='Name')

    # Using Plotly for more customizable visualization
    fig = px.bar(df, x='Name', y='Count', color='Type',
                 hover_data={'Name': True, 'Count': True, 'Type': True},
                 labels={'Count': 'Count', 'Name': 'Observable Name', 'Type': 'Observable Type'})

    fig.show()